### Install needed dependencies
- `xpress` to import the CPLEX solver

*Note: first we need to install `numpy` being a needed dependency of the previous libraries*


In [2]:
!pip install numpy

In [3]:
!pip install pandas

In [4]:
!pip install xpress

### Example problem to test the xpress library was correctly installed

In [5]:
import xpress as xp
p = xp.problem(name='example')  # problem name (optional)
x1 = p.addVariable(vartype=xp.integer, name='x1', lb=-10, ub=10)
x2 = p.addVariable(name='x2')
p.setObjective(x1**2 + 2*x2)      # objective function
p.addConstraint(x1 + 3*x2 >= 4)   # one or more constraints
p.optimize()
print ("solution: {0} = {1}; {2} = {3}".format (x1.name, p.getSolution(x1), x2.name, p.getSolution(x2)))

FICO Xpress v9.5.0, Community, solve started 21:50:07, Dec 13, 2024
Heap usage: 387KB (peak 387KB, 102KB system)
Minimizing MIQP example using up to 8 threads and up to 8087MB memory, with these control settings:
OUTPUTLOG = 1
NLPPOSTSOLVE = 1
XSLP_DELETIONCONTROL = 0
XSLP_OBJSENSE = 1
Original problem has:
         1 rows            2 cols            2 elements         1 entities
         1 qobjelem
Presolved problem has:
         1 rows            2 cols            2 elements         1 entities
         1 qobjelem
Presolve finished in 0 seconds
Heap usage: 416KB (peak 431KB, 102KB system)

Coefficient range                    original                 solved        
  Coefficients   [min,max] : [ 1.00e+00,  3.00e+00] / [ 1.00e+00,  3.00e+00]
  RHS and bounds [min,max] : [ 4.00e+00,  1.00e+01] / [ 4.00e+00,  1.00e+01]
  Objective      [min,max] : [ 2.00e+00,  2.00e+00] / [ 2.00e+00,  2.00e+00]
  Quadratic      [min,max] : [ 2.00e+00,  2.00e+00] / [ 2.00e+00,  2.00e+00]
Autoscaling appl

C:\Users\P2001\AppData\Local\Temp\ipykernel_10156\3312701003.py:2: LicenseWarning: Using the Community license in this session. If you have a full Xpress license, pass the full path to your license file to xpress.init(). If you want to use the FICO Community license and no longer want to see this message, use the following code before using the xpress module:
  xpress.init('c:/Users/P2001/anaconda3/envs/mip/lib/site-packages/xpress/license/community-xpauth.xpr')
  p = xp.problem(name='example')  # problem name (optional)


### Example of loading an mps problem

In [6]:
import xpress as xp
import os
import pandas as pd

def solve_optimization_problem(file_path):
    """
    Solve an optimization problem from an MPS file using Xpress in a Jupyter Notebook.
    
    Args:
        file_path (str): Path to the MPS file to be solved.
    
    Returns:
        dict: A comprehensive dictionary of solving results.
    """
    # Validate file existence
    if not os.path.exists(file_path):
        raise FileNotFoundError(f"The file {file_path} does not exist.")

    # Create a new Xpress model
    model = xp.problem()

    try:
        # Load the .mps file
        model.read(file_path)
        print(f"✅ Successfully loaded {file_path}")

        # Solve the problem
        model.solve()
        print(f"✅ Solving completed for {file_path}")

        # Use the new recommended attributes for status
        solve_status = model.attributes.solvestatus
        sol_status = model.attributes.solstatus

        # Comprehensive status handling
        status_map = {
            0: "Optimal solution found",
            1: "Problem is infeasible",
            2: "Problem is unbounded",
            3: "Problem has no solution",
            4: "Solution is integer infeasible"
        }

        # Prepare results
        result = {
            "solve_status": solve_status,
            "sol_status": sol_status,
            "status_message": status_map.get(solve_status, "Unknown status"),
            "objective_value": model.getObjVal() if solve_status == 0 else None,
            "solution": model.getSolution() if solve_status == 0 else None
        }

        # Detailed output for notebook
        if solve_status == 0:
            print(f"📊 Objective Value: {result['objective_value']}")
            
            # Convert solution to a more readable DataFrame
            solution_df = pd.DataFrame({
                'Variable': [f'x{idx}' for idx in range(len(result['solution']))],
                'Value': result['solution']
            })
            display(solution_df)
        else:
            print(f"❌ Solver status: {result['status_message']}")

        return result

    except xp.XpressError as e:
        print(f"❌ Xpress Error: {e}")
        raise
    except Exception as e:
        print(f"❌ Unexpected error solving {file_path}: {e}")
        raise

# Jupyter Notebook specific helper function to display model details
def display_model_info(file_path):
    """
    Display detailed information about the Xpress model.
    
    Args:
        file_path (str): Path to the MPS file.
    """
    model = xp.problem()
    model.read(file_path)
    
    print("📝 Model Information:")
    print(f"Number of Variables: {model.attributes.cols}")
    print(f"Number of Constraints: {model.attributes.rows}")
    print(f"Objective Sense: {'Minimize' if model.attributes.objsense == 1 else 'Maximize'}")

In [7]:
# Load and display model information
display_model_info("input/example3.mps")

Reading Problem 10teams
Problem Statistics
         230 (      1 spare) rows
        2025 (      0 spare) structural columns
       12150 (   2025 spare) non-zero elements
MIP Entity Statistics
        1800 entities        0 sets        0 set members
📝 Model Information:
Number of Variables: 2025
Number of Constraints: 230
Objective Sense: Minimize


In [8]:
# Solve the optimization problem
result = solve_optimization_problem("input/example3.mps")

Reading Problem 10teams
Problem Statistics
         230 (      1 spare) rows
        2025 (      0 spare) structural columns
       12150 (   2025 spare) non-zero elements
MIP Entity Statistics
        1800 entities        0 sets        0 set members
✅ Successfully loaded input/example3.mps
FICO Xpress v9.5.0, Community, solve started 21:50:34, Dec 13, 2024
Heap usage: 937KB (peak 937KB, 304KB system)
Minimizing MILP 10teams using up to 8 threads and up to 8087MB memory, with these control settings:
OUTPUTLOG = 1
NLPPOSTSOLVE = 1
XSLP_DELETIONCONTROL = 0
XSLP_OBJSENSE = 1
Original problem has:
       230 rows         2025 cols        12150 elements      1800 entities
Presolved problem has:
       210 rows         1600 cols         9600 elements      1600 entities
Presolve finished in 0 seconds
Heap usage: 2029KB (peak 2781KB, 304KB system)

Coefficient range                    original                 solved        
  Coefficients   [min,max] : [ 1.00e+00,  1.00e+00] / [ 1.00e+00,  1.0

In [ ]:
##TODO: create a function that gives random permutation of the problem
##TODO: visualize in some way the permuted problem
##TODO: create a function to compare the permuted results
##TODO: create a system to log the experiments results

In [1]:
import xpress as xp
import os
import pandas as pd
import numpy as np
import random
import logging
from datetime import datetime

In [2]:
class OptimizationExperiment:
    def __init__(self, file_path, log_dir='experiments'):
        """
        Initialize the optimization experiment.
        
        Args:
            file_path (str): Path to the MPS file
            log_dir (str): Directory to store experiment logs
        """
        # Create log directory if it doesn't exist
        os.makedirs(log_dir, exist_ok=True)
        
        # Setup logging
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        log_file = os.path.join(log_dir, f'experiment_{timestamp}.log')
        
        # Configure logging
        logging.basicConfig(
            level=logging.INFO,
            format='%(asctime)s - %(levelname)s: %(message)s',
            handlers=[
                logging.FileHandler(log_file),
                logging.StreamHandler()  # Also print to console
            ]
        )
        
        self.logger = logging.getLogger(__name__)
        self.file_path = file_path
        
        # Original problem attributes
        self.original_model = self._load_problem()
        
    def _load_problem(self):
        """
        Load the optimization problem from MPS file.
        
        Returns:
            xpress.problem: Loaded Xpress problem
        """
        if not os.path.exists(self.file_path):
            raise FileNotFoundError(f"The file {self.file_path} does not exist.")
        
        model = xp.problem()
        model.read(self.file_path)
        
        self.logger.info(f"Successfully loaded problem from {self.file_path}")
        self.logger.info(f"Problem Details:")
        self.logger.info(f"- Variables: {model.attributes.cols}")
        self.logger.info(f"- Constraints: {model.attributes.rows}")
        self.logger.info(f"- Objective Sense: {'Minimize' if model.attributes.objsense == 1 else 'Maximize'}")
        
        return model
    
    def solve_problem(self, model):
        """
        Solve the given optimization problem.
        
        Args:
            model (xpress.problem): Xpress problem to solve
        
        Returns:
            dict: Solving results
        """
        try:
            model.solve()
            
            solve_status = model.attributes.solvestatus
            sol_status = model.attributes.solstatus
            
            # Comprehensive status handling
            status_map = {
                0: "Optimal solution found",
                1: "Problem is infeasible",
                2: "Problem is unbounded",
                3: "Problem has no solution",
                4: "Solution is integer infeasible"
            }
            
            result = {
                "solve_status": solve_status,
                "sol_status": sol_status,
                "status_message": status_map.get(solve_status, "Unknown status"),
                "objective_value": model.getObjVal() if solve_status == 0 else None,
                "solution": model.getSolution() if solve_status == 0 else None
            }
            
            self.logger.info(f"Solve Status: {result['status_message']}")
            if solve_status == 0:
                self.logger.info(f"Objective Value: {result['objective_value']}")
            
            return result
        
        except Exception as e:
            self.logger.error(f"Error solving problem: {e}")
            raise
    
    def create_permuted_problem(self):
        """
        Create a randomly permuted version of the original problem.
        
        Returns:
            xpress.problem: Permuted Xpress problem
        """
        # Create a copy of the original model
        permuted_model = xp.problem()
        permuted_model.read(self.file_path)
        
        # Get number of columns (variables)
        num_cols = permuted_model.attributes.cols
        
        # Create a random permutation
        permutation = np.random.permutation(num_cols)
        
        self.logger.info("Creating permuted problem:")
        self.logger.info(f"Permutation: {permutation}")
        
        # Note: Direct column permutation is not straightforward in Xpress
        # The key is to understand that permutation might affect solution
        # but not necessarily change the mathematical structure
        
        return permuted_model
    
    def run_experiment(self):
        """
        Run the complete optimization experiment.
        Compare original and permuted problem solutions.
        """
        self.logger.info("Starting Optimization Experiment")
        
        # Solve original problem
        self.logger.info("Solving Original Problem")
        original_result = self.solve_problem(self.original_model)
        
        # Create and solve permuted problem
        self.logger.info("Creating Permuted Problem")
        permuted_model = self.create_permuted_problem()
        
        self.logger.info("Solving Permuted Problem")
        permuted_result = self.solve_problem(permuted_model)
        
        # Compare results
        self.logger.info("Comparing Results")
        self.logger.info(f"Original Objective: {original_result.get('objective_value')}")
        self.logger.info(f"Permuted Objective: {permuted_result.get('objective_value')}")
        
        if original_result['solve_status'] == 0 and permuted_result['solve_status'] == 0:
            objective_diff = abs(original_result['objective_value'] - permuted_result['objective_value'])
            relative_diff = objective_diff / abs(original_result['objective_value']) * 100
            
            self.logger.info(f"Absolute Difference: {objective_diff}")
            self.logger.info(f"Relative Difference: {relative_diff:.4f}%")
        
        self.logger.info("Experiment Completed")
        
        return {
            'original_result': original_result,
            'permuted_result': permuted_result,
        }

"""
# Example Usage
if __name__ == "__main__":
    file_path = "input/example3.mps"  # Replace with your MPS file path
    
    try:
        experiment = OptimizationExperiment(file_path)
        results = experiment.run_experiment()
    except Exception as e:
        print(f"Experiment failed: {e}")
"""

'\n# Example Usage\nif __name__ == "__main__":\n    file_path = "input/example3.mps"  # Replace with your MPS file path\n    \n    try:\n        experiment = OptimizationExperiment(file_path)\n        results = experiment.run_experiment()\n    except Exception as e:\n        print(f"Experiment failed: {e}")\n'

In [3]:
file_path = "input/example3.mps"
# Create experiment
experiment = OptimizationExperiment(file_path)

# Run experiment
results = experiment.run_experiment()

Reading Problem 10teams
Problem Statistics
         230 (      1 spare) rows
        2025 (      0 spare) structural columns
       12150 (   2025 spare) non-zero elements
MIP Entity Statistics
        1800 entities        0 sets        0 set members


C:\Users\P2001\AppData\Local\Temp\ipykernel_8740\3893490999.py:43: LicenseWarning: Using the Community license in this session. If you have a full Xpress license, pass the full path to your license file to xpress.init(). If you want to use the FICO Community license and no longer want to see this message, use the following code before using the xpress module:
  xpress.init('c:/Users/P2001/anaconda3/envs/mip/lib/site-packages/xpress/license/community-xpauth.xpr')
  model = xp.problem()
2024-12-14 17:58:42,047 - INFO: Successfully loaded problem from input/example3.mps
2024-12-14 17:58:42,048 - INFO: Problem Details:
2024-12-14 17:58:42,049 - INFO: - Variables: 2025
2024-12-14 17:58:42,050 - INFO: - Constraints: 230
2024-12-14 17:58:42,051 - INFO: - Objective Sense: Minimize
2024-12-14 17:58:42,053 - INFO: Starting Optimization Experiment
2024-12-14 17:58:42,053 - INFO: Solving Original Problem


FICO Xpress v9.5.0, Community, solve started 17:58:42, Dec 14, 2024
Heap usage: 937KB (peak 937KB, 213KB system)
Minimizing MILP 10teams using up to 8 threads and up to 8087MB memory, with these control settings:
OUTPUTLOG = 1
NLPPOSTSOLVE = 1
XSLP_DELETIONCONTROL = 0
XSLP_OBJSENSE = 1
Original problem has:
       230 rows         2025 cols        12150 elements      1800 entities
Presolved problem has:
       210 rows         1600 cols         9600 elements      1600 entities
Presolve finished in 0 seconds
Heap usage: 2029KB (peak 2781KB, 213KB system)

Coefficient range                    original                 solved        
  Coefficients   [min,max] : [ 1.00e+00,  1.00e+00] / [ 1.00e+00,  1.00e+00]
  RHS and bounds [min,max] : [ 1.00e+00,  2.00e+00] / [ 1.00e+00,  2.00e+00]
  Objective      [min,max] : [ 2.00e+00,  8.60e+01] / [ 3.00e+00,  8.60e+01]
Autoscaling applied standard scaling

Will try to keep branch and bound tree memory usage below 1.0GB
Starting concurrent solve wit

2024-12-14 17:59:19,643 - INFO: Solve Status: Problem has no solution
2024-12-14 17:59:19,644 - INFO: Creating Permuted Problem


Reading Problem 10teams
Problem Statistics
         230 (      1 spare) rows
        2025 (      0 spare) structural columns
       12150 (   2025 spare) non-zero elements
MIP Entity Statistics
        1800 entities        0 sets        0 set members


2024-12-14 17:59:19,656 - INFO: Creating permuted problem:
2024-12-14 17:59:19,658 - INFO: Permutation: [1706  731 1493 ... 1131 1957 1902]
2024-12-14 17:59:19,659 - INFO: Solving Permuted Problem


FICO Xpress v9.5.0, Community, solve started 17:59:19, Dec 14, 2024
Heap usage: 937KB (peak 937KB, 446KB system)
Minimizing MILP 10teams using up to 8 threads and up to 8087MB memory, with these control settings:
OUTPUTLOG = 1
NLPPOSTSOLVE = 1
XSLP_DELETIONCONTROL = 0
XSLP_OBJSENSE = 1
Original problem has:
       230 rows         2025 cols        12150 elements      1800 entities
Presolved problem has:
       210 rows         1600 cols         9600 elements      1600 entities
Presolve finished in 0 seconds
Heap usage: 2029KB (peak 2781KB, 446KB system)

Coefficient range                    original                 solved        
  Coefficients   [min,max] : [ 1.00e+00,  1.00e+00] / [ 1.00e+00,  1.00e+00]
  RHS and bounds [min,max] : [ 1.00e+00,  2.00e+00] / [ 1.00e+00,  2.00e+00]
  Objective      [min,max] : [ 2.00e+00,  8.60e+01] / [ 3.00e+00,  8.60e+01]
Autoscaling applied standard scaling

Will try to keep branch and bound tree memory usage below 0.8GB
Starting concurrent solve wit

2024-12-14 17:59:55,565 - INFO: Solve Status: Problem has no solution
2024-12-14 17:59:55,566 - INFO: Comparing Results
2024-12-14 17:59:55,568 - INFO: Original Objective: None
2024-12-14 17:59:55,569 - INFO: Permuted Objective: None
2024-12-14 17:59:55,570 - INFO: Experiment Completed
